In [1]:
import numpy as np
import open3d as o3d
import nibabel as nib
import pyvista
from skimage.measure import marching_cubes
import numpy as np
import time

INFO - 2023-06-24 15:22:26,676 - utils - Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 2023-06-24 15:22:26,677 - utils - NumExpr defaulting to 8 threads.


### Get mesh via marching cube and visualize

In [27]:
# IDs of MRI used for experiments
ids_path = "/vol/space/projects/ukbb/projects/silhouette/eids_filtered.npy"
ids = np.load(ids_path)
ids

array([1000071, 1000180, 1000456, ..., 6023419, 6023786, 6023955])

In [28]:
# masked MRI example for visualization
mri_path = "/vol/space/projects/ukbb/projects/silhouette/labels/v1/3646767/body_mask.nii.gz"

In [29]:
body_segment = nib.load(mri_path)
body_segment_data = body_segment.get_fdata()
# create mesh from masked MRI image
verts, faces, norms, vals = marching_cubes(body_segment_data, level=0, step_size=1) 
verts = verts/np.array(body_segment_data.shape) 
edges = np.concatenate((faces[:,:2], faces[:,1:]), axis=0)

In [30]:
faces.shape

(511206, 3)

In [31]:
verts.shape

(255689, 3)

In [32]:
edges.shape

(1022412, 2)

In [33]:
# visualize original mesh
mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(np.asarray(verts)),
                                 triangles=o3d.utility.Vector3iVector(np.asarray(faces)))

# uncomment out the following 2 lines for color-rendered visualization
# original_mesh_paint = np.asarray([0,200,220])/255.0
# mesh.paint_uniform_color(original_mesh_paint)

mesh.compute_vertex_normals()
mesh.compute_convex_hull()
o3d.visualization.draw_geometries([mesh],mesh_show_back_face=True,mesh_show_wireframe=False)


[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


### decimate mesh and visualize

In [35]:
# decimate mesh according to number of faces
decimation_level = 1000 # how many faces left after decimation
mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(np.asarray(verts)),
                                 triangles=o3d.utility.Vector3iVector(np.asarray(faces)))


start = time.time()
decimated_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(mesh, decimation_level)
end = time.time()

print("The time of execution of above program is :",(end-start) , "s")
print("The time of execution of above program is :",(end-start) * 10**3, "ms")

# visualize 

# uncomment out the following 2 lines for color-rendered visualization
# decimated_mesh_paint = np.asarray([230,200,110])/255.0
# decimated_mesh.paint_uniform_color(decimated_mesh_paint)
decimated_mesh.compute_vertex_normals()
decimated_mesh.compute_convex_hull()
o3d.visualization.draw_geometries([decimated_mesh],mesh_show_back_face=True,mesh_show_wireframe=False)

The time of execution of above program is : 7.822888612747192 s
The time of execution of above program is : 7822.888612747192 ms
[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


## Registration

In [36]:
TARGET_SAMPLE = "/vol/space/projects/ukbb/projects/silhouette/decimated_5/4266049.ply"
data_path = "/vol/space/projects/ukbb/projects/silhouette/decimated_5/3946754.ply"
THRESHOLD = 0.02

In [37]:
target_trm = o3d.io.read_triangle_mesh(TARGET_SAMPLE)
target_trm.compute_vertex_normals()
target_pcd = o3d.geometry.PointCloud(points = target_trm.vertices)
target_pcd.estimate_normals()

In [38]:
start = time.time()
# reading source as triangle mesh
source_trm = o3d.io.read_triangle_mesh(data_path)
# creating a point cloud for ICP from triangle mesh
source_pcd = o3d.geometry.PointCloud(points = source_trm.vertices)
source_pcd.estimate_normals()
# identity transformation
trans_init = np.identity(4) 
reg_p2l = o3d.pipelines.registration.registration_icp(source_pcd, target_pcd, THRESHOLD, trans_init, o3d.pipelines.registration.TransformationEstimationPointToPlane())

# apply transformation to source point cloud
source_pcd.transform(reg_p2l.transformation)

# create new triangle mesh object from transformed source point cloud and source triangle mesh face data
new_mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(np.asarray(source_pcd.points)),
                                        triangles=o3d.utility.Vector3iVector(np.asarray(source_trm.triangles)))
new_mesh.compute_vertex_normals()

end = time.time()

print("The time of execution of above program is :",(end-start) , "s")
print("The time of execution of above program is :",(end-start) * 10**3, "ms")


The time of execution of above program is : 0.047873497009277344 s
The time of execution of above program is : 47.873497009277344 ms


In [6]:
mesh1k = o3d.io.read_triangle_mesh('/vol/space/projects/ukbb/projects/silhouette/registered_1/2784128.ply')
mesh5k = o3d.io.read_triangle_mesh('/vol/space/projects/ukbb/projects/silhouette/decimated_5/2784128.ply')
mesh10k = o3d.io.read_triangle_mesh('/vol/space/projects/ukbb/projects/silhouette/decimated_10/2784128.ply')
mesh25k = o3d.io.read_triangle_mesh('/vol/space/projects/ukbb/projects/silhouette/decimated_25/2784128.ply')

In [7]:
num_faces1k = len(mesh1k.triangles)
num_faces5k = len(mesh5k.triangles)
num_faces10k = len(mesh10k.triangles)
num_faces25k = len(mesh25k.triangles)

In [8]:
num_faces1k

1000

In [5]:
num_faces5k

4999

In [8]:
num_faces10k

10000

In [9]:
num_faces25k

24999